In [1]:
import pandas as pd
import re

In [14]:
from google.colab import files

file_contents = None  # Variable to store file content

def handle_file_upload(change):
    global file_contents
    uploaded = upload_button.value
    for filename in uploaded.keys():
        file_contents = uploaded[filename]['content'].decode('utf-8')
        print(f"Uploaded '{filename}' with length {len(file_contents)} bytes")
        print("File Contents:\n", file_contents)

import ipywidgets as widgets

upload_button = widgets.FileUpload()
upload_button.observe(handle_file_upload, names='value')

display(upload_button)

FileUpload(value={}, description='Upload')

In [ ]:
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s*\d{1,2}:\d{2}\s*[APMapm]+ - '
message = re.split(pattern, file_contents)[1:]
#print(message)
dates=re.findall(pattern, file_contents)

df=pd.DataFrame({'user_message':message,'message_date':dates})
df.rename(columns={'message_date':'date'},inplace=True)

#print(df.head())
user=[]
messages=[]
for message in df['user_message']:
    entry=re.split('([\w\W]+?):\s', message)
    if entry[1:]:
      if entry[2] != 'This message was deleted\n' and entry[2]!="<Media omitted>\n":
        user.append(entry[1])
        str1=entry[1]
        str2=entry[2]
        res=str1+" said "+str2
        messages.append(res)
      else:
        user.append('group_notification')
        messages.append(entry[2])
    else:
        user.append('group_notification')
        messages.append(entry[0])
df['user']=user
df['message']=messages
df.drop(columns=['user_message'],inplace=True)

print(df.head())

In [ ]:
n = int(input("Enter the number of recent chats to be summarized: "))
num = len(messages) - n
msg = []

for i in range(num, len(messages)):
    if user[i] != 'group_notification':
        str1 = [char for char in messages[i]]
        str1[-1] = '.'
        msg.append(str1)

flat_msg = [char for sublist in msg for char in sublist]

text = "".join(flat_msg)

print(text)



In [ ]:
import nltk
import networkx as nx
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    sentences = sent_tokenize(text)
    stop_words = set(stopwords.words('english'))
    sentence_tokens = [word_tokenize(sentence.lower()) for sentence in sentences]
    sentence_tokens = [[word for word in tokens if word.isalnum() and word not in stop_words] for tokens in sentence_tokens]
    return sentence_tokens

def calculate_similarity(sentence1, sentence2):
    words1 = set(sentence1)
    words2 = set(sentence2)
    intersection = words1.intersection(words2)
    union = words1.union(words2)
    return len(intersection) / len(union) if len(union) > 0 else 0

def build_similarity_matrix(sentences):
    sentence_tokens_array = np.array(sentences)

    similarity_matrix = [[calculate_similarity(sentence1, sentence2) for sentence2 in sentence_tokens_array] for sentence1 in sentence_tokens_array]
    return similarity_matrix

def textrank_summarize(text, num_lines=10):
    sentence_tokens = preprocess_text(text)
    similarity_matrix = build_similarity_matrix(sentence_tokens)

    graph = nx.from_numpy_array(np.array(similarity_matrix))

    scores = nx.pagerank(graph)

    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentence_tokens)), reverse=True)

    top_sentences = [sentence for score, sentence in ranked_sentences]

    summary = ' '.join([' '.join(sentence) for sentence in top_sentences[:num_lines]])
    return summary

summary = textrank_summarize(text, num_lines=10)
print(summary)
